In [1]:
#Importing The Libraries
from skimage.filters import threshold_local
import numpy as np
import cv2
import imutils #Resize and Cropping Images
import argparse as ar

In [2]:
# Constructing Argument parser
args = {"image": "receipt.jpg"}
print(args)

{'image': 'receipt.jpg'}


In [3]:
## Edge Detection
# Load the image and compute the ratio of old height to new height clone it and resize it
image = cv2.imread(args["image"])

# To speed up image processing we resize the image to 500 pixel
ratio = image.shape[0]/500.0
original = image.copy()
image = imutils.resize(image, height = 500)

# Converting the images to gray scale and findinf=g the edges
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
gray = cv2.GaussianBlur(gray , (5,5),0)
edged = cv2.Canny(gray , 75 , 200)

# Output the original image and edge detected image
print(" Step 1 For Scanning Images - Edge Detection")
cv2.imshow("Original Image", image)
cv2.imshow("Edge Iamge", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

 Step 1 For Scanning Images - Edge Detection


In [4]:
# Asumme the piece of paper is main focus - Finding the Contour of the edged image keeping only large ones
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST , cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

# Loop over contour
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
    
    # If Contour has 4 points we have find out our piece of screen
    if len(approx) == 4:
        screenCnt = approx
        break
        
# Show the contour of the piece of paer
print(" Step 2 for Finding contour of image")
cv2.drawContours(image, [screenCnt], -1, (0,255,0), 2)
cv2.imshow("Outline Of iamge", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

 Step 2 for Finding contour of image


In [5]:
# import the necessary packages
import numpy as np
import cv2
 
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
 
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
 
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
 
    # return the ordered coordinates
    return rect

In [6]:
def four_point_transforms(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
 
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
 
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
 
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
 
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
 
    # return the warped image
    return warped

In [7]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transforms(original, screenCnt.reshape(4, 2) * ratio)
 
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
 
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(original, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

STEP 3: Apply perspective transform


32